# Load libraries & helper functions

In [ ]:
%matplotlib inline
from helpers import *
# Turn interactive plotting off
plt.ioff()

# Declare parameters

In [ ]:
config.DATADIR      = '/home/duboisjx/vault/data/CamCAN/MRI'
config.fmriRun      = 'task-Movie_bold'
# use volume or surface data
config.isCifti      = False

config.pipelineName = 'Finn'
# use ICA-FIX as starting point
config.useFIX       = False
config.preWhitening = False
config.Operations   = config.operationDict[config.pipelineName]

config.overwrite    = False

# submit jobs with sge
config.queue        = False
# make sure to set memory requirements according to data size
# 15G for HCP data!
config.maxvmem      = '4G'

# parcellation for FC matrix
config.parcellationName = 'shen2013'
config.parcellationFile = '/home/duboisjx/vault/data/parcellations/shenetal_neuroimage2013_new/shen_3mm_268_parcellation.nii.gz'
config.nParcels         = 268

subjects = [d for d in listdir(config.DATADIR) if d[0:6]=="sub-CC"]
subjects.sort()
print 'Found {} subjects'.format(len(subjects))
#print subjects

# Do work

In [ ]:
keep      = np.zeros((len(subjects)),dtype=np.bool_)
fdScores  = np.zeros((len(subjects)),dtype=np.float32)
fcMats    = np.zeros((config.nParcels,config.nParcels,len(subjects)),dtype=np.float32)
fcMats_dn = np.zeros((config.nParcels,config.nParcels,len(subjects)),dtype=np.float32)
i=0
displayPlot=False
for config.subject in subjects:
    print 'SUB {}/{}: {}'.format(i+1,len(subjects),config.subject)
    keep[i] = runPipelinePar()
    if keep[i]:
        fcMats[:,:,i],fcMats_dn[:,:,i] = plotFC(displayPlot)
        fdScores[i]=np.mean(computeFD())
    i = i + 1
print 'Using {}/{} subjects'.format(np.sum(keep),len(subjects))
fdScores  = fdScores[keep]
fcMats    = fcMats[:,:,keep]
fcMats_dn = fcMats_dn[:,:,keep]

# rsFC/mean FD plot
# since this is a correlation across subjects, it makes most sense if we have enough subjects
if len(subjects)>20:    
    plotQCrsFC(fcMats,fcMats_dn,fdScores)
    
# deltaR plot
plotDeltaR(fcMats,fcMats_dn)
